In [1]:
import requests
import pandas as pd
from env import api_key
from datetime import datetime

### /v8/finance/spark - historical data for various intervals and ranges

In [ ]:
# Grabbing 5 years of close prices for Microsoft and Apple
headers = {
    'x-api-key': api_key
    }

response = requests.request("GET", 'https://yfapi.net/v8/finance/spark?interval=1d&range=5y&symbols=AAPL,MSFT', headers=headers)

In [3]:
# Saving response to data 
data = response.json()

In [76]:
def extract_date_and_price(data):
    '''
    This function takes in JSON file,
    creates a series for price and date,
    converts timestamp into a proper date,
    combines both price and date series into one dataframe,
    returns the dataframe
    '''
    # Creating an empty DataFrame
    df = pd.DataFrame()

    # Looping through each symbol in data
    for symbol in data:

        # Creating a series for timestamp and closing price
        timestamp = pd.Series(data[symbol]['timestamp'])
        price = pd.Series(data[symbol]['close'])

        # Converting timestamp to year/month/day format
        timestamp = timestamp.apply(lambda x : datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))

        # Saving series into a the empty DataFrame I created earlier
        df[symbol + '_price'] = price
        df[symbol + '_date'] = timestamp
        
    return df

In [29]:
# Example output of just two symbols using the extract function
df

,MSFT_price,MSFT_date,AAPL_price,AAPL_date
0,62.30,2017-01-04,29.005,2017-01-04
1,62.30,2017-01-05,29.153,2017-01-05
2,62.84,2017-01-06,29.478,2017-01-06
3,62.64,2017-01-09,29.747,2017-01-09
4,62.62,2017-01-10,29.778,2017-01-10
...,...,...,...,...
1254,341.25,2021-12-28,179.290,2021-12-28
1255,341.95,2021-12-29,179.380,2021-12-29
1256,339.32,2021-12-30,178.200,2021-12-30
1257,336.32,2021-12-31,177.570,2021-12-31


###  Grabbing the top 250 day_gainers

In [58]:
url = 'https://yfapi.net/ws/screeners/v1/finance/screener/predefined/saved?count=250&scrIds=day_gainers'

headers = {
    'x-api-key': api_key
    }

response = requests.request("GET", url, headers=headers)
data = response.json()

In [59]:
# Inspecting the Data
data

{'finance': {'result': [{'id': 'ec5bebb9-b7b2-4474-9e5c-3e258b61cbe6',
    'title': 'Day Gainers - US',
    'description': 'Stocks ordered in descending order by price percent change greater than 3% with respect to the previous close',
    'canonicalName': 'DAY_GAINERS',
    'criteriaMeta': {'size': 250,
     'offset': 0,
     'sortField': 'percentchange',
     'sortType': 'DESC',
     'quoteType': 'EQUITY',
     'topOperator': 'AND',
     'criteria': [{'field': 'percentchange',
       'operators': ['GT'],
       'values': [3.0],
       'labelsSelected': []},
      {'field': 'region',
       'operators': ['EQ'],
       'values': [],
       'labelsSelected': [52]},
      {'field': 'intradaymarketcap',
       'operators': ['EQ'],
       'values': [],
       'labelsSelected': [1, 2, 3]},
      {'field': 'dayvolume',
       'operators': ['GT'],
       'values': [15000],
       'labelsSelected': []}]},
    'rawCriteria': '{"offset":0,"size":250,"sortField":"percentchange","sortType":"DESC",

In [72]:
# Creating an empty list to grab the symbols 
list_of_symbols = []

# Going to be looping through the list of quotes
quotes = data['finance']['result'][0]['quotes']

# Loop through quotes and save symbol to list
for quote in quotes:
    list_of_symbols.append(quote['symbol'])

In [125]:
# Trying to automate grabbing the price of the most popular 250 stocks for the last 5 years
headers = {
    'x-api-key': api_key
    }


# Testing out only the first 10 stocks of list_of_stocks
for y in range(10, 20, 10):
    
    # Setting x to 10 less than y
    x = y-10
    
    # Created an emtpy string
    placeholder = ""
    
    # Creating a placeholder string from list of symbols
    for index, symbol in enumerate(list_of_symbols[x:y]):
        
        # if index is equal to nine then this is last symbol and doesn't require a comma at the end
        if index == 9:
            placeholder += symbol
            
        # All other index's will require a comma
        else:
            placeholder += symbol + ","
            
    # Using requests to get the last 5 years of the 5 stocks saved under placeholder
    response = requests.request("GET", f'https://yfapi.net/v8/finance/spark?interval=1d&range=5y&symbols={placeholder}', headers=headers)
    data = response.json()
    
    # Using the function defined above to create a Data Frame
    df = extract_date_and_price(data)

In [126]:
# Example out of 10 stocks, ran into a problem of NaNs
df

,RIG_price,RIG_date,TSLA_price,TSLA_date,FCEL_price,FCEL_date,DLAKF_price,DLAKF_date,FMCC_price,FMCC_date,EBS_price,EBS_date,LILM_price,LILM_date,IBRX_price,IBRX_date,STNE_price,STNE_date,AAGH_price,AAGH_date
0,15.450,2017-01-04,45.398,2017-01-04,22.8,2017-01-04,13.16,2017-01-04,3.97,2017-01-04,33.88,2017-01-04,10.54,2020-11-23,6.08,2017-01-04,31.35,2018-10-25,0.0060,2017-01-04
1,15.620,2017-01-05,45.350,2017-01-05,24.0,2017-01-06,13.16,2017-01-11,3.84,2017-01-05,33.31,2017-01-05,10.60,2020-11-24,6.34,2017-01-05,31.09,2018-10-26,0.0060,2017-01-11
2,15.770,2017-01-06,45.802,2017-01-06,23.4,2017-01-09,12.16,2017-01-12,3.86,2017-01-06,34.66,2017-01-06,10.65,2020-11-25,6.51,2017-01-06,29.06,2018-10-29,0.0052,2017-01-12
3,15.290,2017-01-09,46.256,2017-01-09,23.4,2017-01-11,12.16,2017-01-24,3.86,2017-01-09,32.04,2017-01-09,11.00,2020-11-27,6.62,2017-01-09,28.68,2018-10-30,0.0052,2017-01-19
4,15.140,2017-01-10,45.974,2017-01-10,22.8,2017-01-12,13.26,2017-01-25,3.84,2017-01-10,31.09,2017-01-10,11.31,2020-11-30,6.56,2017-01-10,28.69,2018-10-31,0.0070,2017-01-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,2.860,2021-12-29,926.920,2021-12-16,NaN,NaN,NaN,NaN,NaN,NaN,42.53,2021-12-16,NaN,NaN,6.20,2021-12-29,NaN,NaN,NaN,NaN
1247,2.840,2021-12-30,932.570,2021-12-17,NaN,NaN,NaN,NaN,NaN,NaN,45.04,2021-12-17,NaN,NaN,6.38,2021-12-30,NaN,NaN,NaN,NaN
1248,2.760,2021-12-31,899.940,2021-12-20,NaN,NaN,NaN,NaN,NaN,NaN,44.36,2021-12-20,NaN,NaN,6.08,2021-12-31,NaN,NaN,NaN,NaN
1249,3.120,2022-01-03,938.530,2021-12-21,NaN,NaN,NaN,NaN,NaN,NaN,45.82,2021-12-21,NaN,NaN,7.53,2022-01-03,NaN,NaN,NaN,NaN


### Ran into the issue of some stocks not being at least 5 years old
    - Need to use the date as the index and line up stocks prices to that index/date.